# <font style = "color:rgb(50,120,229)">Delaunay Triangulation and Voronoi Diagram</font>
In a previous lecture we learned how approximating a nonlinear warp with piecewise linear map can lead to huge gains in efficiency. The next obvious question is how to generate these non-overlapping triangles that cover the entire image. 

One easy option is to divide the image into large squares and then split every square into two triangles. That is adequate for many applications. 

In the applications we are going to discuss, like Face Morphing, Face Averaging and Face Swapping, we need to warp corresponding areas of two different faces. In such cases, the triangulation needs to be based on the the facial landmark points. 

## <font style = "color:rgb(50,120,229)">What is Delaunay Triangulation?</font>

&nbsp;

<center><a href="https://www.learnopencv.com/wp-content/uploads/2017/12/opcv4face-w4-m3-dealunyTriangle-1.png"><img src = "https://www.learnopencv.com/wp-content/uploads/2017/12/opcv4face-w4-m3-dealunyTriangle-1.png"/></a></center>

**<center>Figure 1. Left : The image with input points shown as red dots. Right: The white lines display Delaunay triangulation calculated using the points. </center>**

Given a set of points in a plane, a triangulation refers to the subdivision of the plane into triangles, with the points as vertices. In Figure 1, we see a set of points on the left image shown using red dots and the triangulation shown using white lines. 

A set of points can have many possible triangulations, but Delaunay triangulation stands out because it has some nice properties. 

In a Delaunay triangulation, triangles are chosen such that **no point is inside the circumcircle of any triangle**. Figure 2. shows Delaunay triangulation of 4 points A, B, C and D.  The difference between the top and the bottom images is that in the top image, point C is further away from line BD as compared to the bottom image. This movement of point C changes the Delaunay triangulation. 

In the top image, for the triangulation to be a valid Delaunay triangulation, point C should be outside the circumcircle of triangle ABD, and point A should be outside the circumcircle of triangle BCD.

<center> <a href="https://www.learnopencv.com/wp-content/uploads/2017/12/opcv4face-w4-m3-angles.png"><img src = "https://www.learnopencv.com/wp-content/uploads/2017/12/opcv4face-w4-m3-angles.png"/></a></center>
<center>**Figure 2 : Delaunay triangulation favors small angles**</center>

&nbsp;
An interesting property of Delaunay triangulation is that it does not favor **"skinny"** triangles ( i.e. triangles with one large angle ).

Figure 2 shows how the triangulation changes to pick "fat" triangles when the points are moved. In the top image, the points B and D have their x-coordinates at x = 1.5, and in the bottom image they are moved to the right to x = 1.75. In the top image angles ABC and ABD are large, and Delaunay triangulation creates an edge between B and D splitting the two large angles into smaller angles ABD, ADB, CDB, and CBD. On the other hand in the bottom image, the angle BCD is too large, and Delaunay triangulation creates an edge AC to divide the large angle.

There are many algorithms to find the Delaunay triangulation of a set of points. The most obvious ( but not the most efficient ) one is to start with any triangulation, and check if the circumcircle of any triangle contains another point. If it does, flip the edges ( as shown in Figure 2. ) and continue until there are no triangles whose circumcircle contains a point.

Any discussion on Delaunay triangulation has to include Voronoi diagrams because the Voronoi diagram of a set of points is mathematical dual to its Delaunay triangulation.

## <font style = "color:rgb(50,120,229)">What is a Voronoi Diagram ?</font>

&nbsp;

<center> <a href="https://www.learnopencv.com/wp-content/uploads/2017/12/opcv4face-w4-m3-voronoiDiagram.png"><img src = "https://www.learnopencv.com/wp-content/uploads/2017/12/opcv4face-w4-m3-voronoiDiagram.png"/></a></center>


**<center>Figure 3: Left : The image with input points shown as red dots. Right: The Voronoi diagram calculated using the input points.</center>** 


Given a set of points in a plane, a Voronoi diagram partitions the space such that the boundary lines are **equidistant** from neighboring points. Figure 3. shows an example of a Voronoi diagram. You will notice that every boundary line passes through the center of two points. 

> <font style="font-family:Poiret one" size="+2"> If you connect the points in neighboring Voronoi regions, you get a Delaunay triangulation!</font>

Given a set of points, you can calculate the Delaunay Triangulation or Voronoi Diagram using the class **Subdiv2D**. Here are the steps. A complete working example is shown in the next section.

## <font style = "color:rgb(50,120,229)">Delaunay Triangulation Code & Tutorial</font>

We will first show a simple code that reads in a file containing points and writes a file containing delaunay triangulation. Each row of the output file contains three numbers corresponding to the three indices of the points that form the triangle. 

In [1]:
#include "includeLibraries.h"
#include <opencv2/opencv.hpp>
#include "matplotlibcpp.h"
#include "displayImages.h"

In [2]:
using namespace std;

In [3]:
using namespace cv;

In [4]:
using namespace matplotlibcpp;

The function `findIndex` finds the index of the closest landmark point to the given vertex from the vector of points.

In [5]:
// In a vector of points, find the index of point closest to input point.
static int findIndex(vector<Point2f>& points, Point2f &point)
{
  int minIndex = 0;
  double minDistance = norm(points[0] - point);
  for(int i = 1; i < points.size(); i++)
  {
    double distance = norm(points[i] - point);
    if( distance < minDistance )
    {
      minIndex = i;
      minDistance = distance;
    }

  }
  return minIndex;
}

The function `writeDelaunay` gets the Delaunay triangles using the `getTriangleList` method of the `subdiv` object.

`getTriangleList` returns a vector of 6 floating point numbers `vector<Vec6f>`. The six numbers are the x and y coordinates of the three vertices of a triangle.

In [6]:
// Write delaunay triangles to file
static void writeDelaunay(Subdiv2D& subdiv, vector<Point2f>& points, const string &filename)
{

  // Open file for writing
  std::ofstream ofs;
  ofs.open(filename);

  // Obtain the list of triangles.
  // Each triangle is stored as vector of 6 coordinates
  // (x0, y0, x1, y1, x2, y2)
  vector<Vec6f> triangleList;
  subdiv.getTriangleList(triangleList);

  // Will convert triangle representation to three vertices
  vector<Point2f> vertices(3);

  // Loop over all triangles
  for( size_t i = 0; i < triangleList.size(); i++ )
  {
    // Obtain current triangle
    Vec6f t = triangleList[i];

    // Extract vertices of current triangle
    vertices[0] = Point2f(t[0], t[1]);
    vertices[1] = Point2f(t[2], t[3]);
    vertices[2] = Point2f(t[4], t[5]);

    // Find landmark indices of vertices in the points list
    int landmark1 = findIndex(points, vertices[0]);
    int landmark2 = findIndex(points, vertices[1]);
    int landmark3 = findIndex(points, vertices[2]);
    // save to file.

    ofs << landmark1 << " " << landmark2 << " " << landmark3 << endl;

  }
  ofs.close();
}

First, we read a data file containing points. Each row of the file contains the x and y coordinates of a point separated by a space.

In [7]:
// Create a vector of points.
vector<Point2f> points;

// Read in the points from a text file
string pointsFilename(DATA_PATH + "images/smiling-man-delaunay.txt");
ifstream ifs(pointsFilename);
int x, y;
while(ifs >> x >> y)
{
    points.push_back(Point2f(x,y));
}

cout << "Reading file " << pointsFilename << endl;

Reading file ../data/images/smiling-man-delaunay.txt


Delaunay triangulation and voronoi diagrams are calculated using the class `Subdiv2D`. 

`Subdiv2D` is initialized with a rectangle containing all the points. The landmark points are read from the file.

The insert method is used to insert points. 

`writeDelaunay` function uses `getTriangleList` method to get the delaunay triangles.

In [8]:
// Find bounding box enclosing the points.
Rect rect = boundingRect(points);

// Create an instance of Subdiv2D
Subdiv2D subdiv(rect);

// Insert points into subdiv
for( vector<Point2f>::iterator it = points.begin(); it != points.end(); it++)
{
    subdiv.insert(*it);
}

Save output Delaunay triangles to file. The function `writeDelaunay` takes as input an instance of `Subdiv2D`, the list of input points, and saves it to disk.

Each row of the output file contains three number that represent a triangle. The numbers are the indices of points in the input file.

In [9]:
// Output filename
string outputFileName("results/smiling-man-delaunay.tri");

// Write delaunay triangles
writeDelaunay(subdiv, points, outputFileName);

cout << "Writing Delaunay triangles to " << outputFileName << endl;

Writing Delaunay triangles to results/smiling-man-delaunay.tri


## <font style = "color:rgb(50,120,229)">Delaunay Triangulation Animation Code & Tutorial</font>

It is instructive to see an animation of Delaunay triangulation and Voronoi diagram as the points get added. Let’s dive into the code. 

The function `drawPoint` draws a point on an image using a specified color.

In [10]:
static void drawPoint( Mat& img, Point2f fp, Scalar color )
{
  circle( img, fp, 2, color, FILLED, LINE_AA, 0 );
}

The function `drawDelaunay` draws Delaunay triangles on an image using a specified color.

Points are inserted using an instance of `Subdiv2D` before calling this function. Delaunay triangles are obtained by calling the method `getTriangleList`.

Once we get the triangles, we draw the lines by going over each vertex of the triangle.

In [11]:
// Draw delaunay triangles
static void drawDelaunay( Mat& img, Subdiv2D& subdiv, Scalar delaunayColor )
{
  // Obtain the list of triangles.
  // Each triangle is stored as vector of 6 coordinates
  // (x0, y0, x1, y1, x2, y2)
  vector<Vec6f> triangleList;
  subdiv.getTriangleList(triangleList);
  
  // Will convert triangle representation to three vertices
  vector<Point> vertices(3);
  
  // Get size of the image
  Size size = img.size();
  Rect rect(0,0, size.width, size.height);
  
  for( size_t i = 0; i < triangleList.size(); i++ )
  {
    // Get current triangle
    Vec6f t = triangleList[i];
    
    // Convert triangle to vertices
    vertices[0] = Point(cvRound(t[0]), cvRound(t[1]));
    vertices[1] = Point(cvRound(t[2]), cvRound(t[3]));
    vertices[2] = Point(cvRound(t[4]), cvRound(t[5]));
    
    // Draw triangles that are completely inside the image.
    if ( rect.contains(vertices[0]) && rect.contains(vertices[1]) && rect.contains(vertices[2]))
    {
      line(img, vertices[0], vertices[1], delaunayColor, 1, LINE_AA, 0);
      line(img, vertices[1], vertices[2], delaunayColor, 1, LINE_AA, 0);
      line(img, vertices[2], vertices[0], delaunayColor, 1, LINE_AA, 0);
    }
  }
}

The function `drawVoronoi` draws Voronoi diagrams on an image.

Points are inserted using an instance of `Subdiv2D` before calling this function. The Voronoi diagram is represented as a collection of facets (polygons) which can be obtained using the the method `getVoronoiFacetList`.

The facets so obtained are drawn on the image using `fillConvexPoly` and the boundaries of the facets are drawn using polylines.

In [12]:
//Draw voronoi diagrams
static void drawVoronoi( Mat& img, Subdiv2D& subdiv )
{
  // Vector of voronoi facets.
  vector<vector<Point2f> > facets;
  
  // Voronoi centers
  vector<Point2f> centers;
  
  // Get facets and centers
  subdiv.getVoronoiFacetList(vector<int>(), facets, centers);
  
  // Variable for the ith facet used by fillConvexPoly
  vector<Point> ifacet;
  
  // Variable for the ith facet used by polylines.
  vector<vector<Point> > ifacets(1);
  
  for( size_t i = 0; i < facets.size(); i++ )
  {
    // Extract ith facet
    ifacet.resize(facets[i].size());
    for( size_t j = 0; j < facets[i].size(); j++ )
      ifacet[j] = facets[i][j];
    
    // Generate random color
    Scalar color;
    color[0] = rand() & 255;
    color[1] = rand() & 255;
    color[2] = rand() & 255;
    
    // Fill facet with a random color
    fillConvexPoly(img, ifacet, color, 8, 0);
    
    // Draw facet boundary
    ifacets[0] = ifacet;
    polylines(img, ifacets, true, Scalar(), 1, LINE_AA, 0);
    
    // Draw centers.
    circle(img, centers[i], 3, Scalar(), FILLED, LINE_AA, 0);
  }
}

The function findIndex and writeDelaunay were discussed above.

In [13]:
// Define colors for drawing.
Scalar delaunayColor(255,255,255), pointsColor(0, 0, 255);

In [14]:
// Read in the image.
Mat img = imread(DATA_PATH + "images/smiling-man.jpg");

In [15]:
// Rectangle to be used with Subdiv2D
Size size = img.size();
Rect rect2(0, 0, size.width, size.height);

The image bounding box is used to initialize an instance of `Subdiv2D`. The points are subsequently added to the this object for calculating and displaying Delaunay triangulation and Voronoi diagrams.

In [16]:
// Create an instance of Subdiv2D
Subdiv2D subdiv(rect2);

// Create a vector of points.
vector<Point2f> points2;

// Read in the points from a text file
ifstream ifs2(DATA_PATH + "images/smiling-man-delaunay.txt");
int x2, y2;
while(ifs2 >> x2 >> y2)
{
    points2.push_back(Point2f(x2,y2));
}

// Image for displaying Delaunay Triangulation
Mat imgDelaunay;

// Image for displaying Voronoi Diagram.
Mat imgVoronoi = Mat::zeros(img.rows, img.cols, CV_8UC3);

// Final side-by-side display.
Mat imgDisplay;

We calculate and draw the Delaunay triangulation ( on `imgDelaunay` ) and Voronoi Diagram on ( `imgVoronoi` ) every time a new point is added thus creating an animation.

In [17]:
// Insert points into subdiv and animate
for( vector<Point2f>::iterator it = points.begin(); it != points.end(); it++)
{
    subdiv.insert(*it);

    imgDelaunay = img.clone();
    imgVoronoi = cv::Scalar(0,0,0);

    // Draw delaunay triangles
    drawDelaunay( imgDelaunay, subdiv, delaunayColor );

    // Draw points
    for( vector<Point2f>::iterator it = points.begin(); it != points.end(); it++)
    {
      drawPoint(imgDelaunay, *it, pointsColor);
    }

    // Draw voronoi map
    drawVoronoi(imgVoronoi, subdiv);

    hconcat(imgDelaunay, imgVoronoi, imgDisplay);
    //imshow(win, imgDisplay);
    //waitKey(100);
}

In [18]:
// Write delaunay triangles
writeDelaunay(subdiv, points, "smiling-man-delaunay.tri");

## <font style = "color:rgb(50,120,229)">Results</font>
If you run the code - delaunayAnimation.cpp in week2-cpp/cpp folder, you should get a result similar to the one shown below.

![](https://www.learnopencv.com/wp-content/uploads/2018/01/cv4f-m4-w2-animation.gif)

# <font style = "color:rgb(50,120,229)">References and Further Reading</font>

[https://en.wikipedia.org/wiki/Delaunay_triangulation](https://en.wikipedia.org/wiki/Delaunay_triangulation)